# Metadata extraction using Meteor

This notebook attempts to extract metadata from the FinGreyLit test set documents using [Meteor](https://github.com/NationalLibraryOfNorway/meteor), an open source metadata extraction tool developed by the National Library of Norway.

It assumes that Meteor is installed and running locally on http://127.0.0.1:5000/ (the default).

In [1]:
import glob
import json

metadata_files = glob.glob("../../metadata/*.jsonl")  # train + test (used for refining code & fixing dataset)
#metadata_files = glob.glob("../../metadata/*-test.jsonl")  # test set only (used for final evaluation)
records = []

for mdfile in metadata_files:
    with open(mdfile) as inf:
        for line in inf:
            rec = json.loads(line)
            records.append(rec)

print(len(records))

725


In [2]:
%%time

import os
import requests

METEOR_API_URL = 'http://127.0.0.1:5000/json'

def id_to_fn(identifier):
    """convert a URI identifier to a simpler string we can use as a filename for the PDF"""
    return '../../pdfs/' + identifier.replace('https://', '').replace('/','_') + ".pdf"

def download(file_url, identifier):
    """download a PDF file, with the given identifier, from the given URL (unless this was done already)
    and return a path to the PDF file"""
    path = id_to_fn(identifier)
    if os.path.exists(path) and os.path.getsize(path) > 0:
        return path

    response = requests.get(file_url)
    with open(path, "wb") as f:
        f.write(response.content)
        print(f"wrote {file_url} as {path}")
    return path

out_records = []

for rec in records:
    path = download(rec['url'], rec['id'])

    # Create a dictionary containing the file to be sent
    filedata = {'fileInput': (path, open(path, 'rb'), 'application/pdf')}

    # Send the POST request with the file
    response = requests.post(METEOR_API_URL, files=filedata)
    try:
        rec['meteor_output'] = response.json()
    except requests.JSONDecodeError:
        print(f"JSON decode error for {rec['id']} / {rec['url']}")
        rec['meteor_output'] = {}
    out_records.append(rec)

# write output to JSONL file
with open('test-records-meteor.jsonl', 'w') as outfile:
    for rec in out_records:
        json.dump(rec, outfile)
        outfile.write("\n")

JSON decode error for https://www.doria.fi/handle/10024/181710 / https://www.doria.fi/bitstream/handle/10024/181710/Laura Hollsten ÅBO AKADEMI OCH KUNSKAPEN doria 23.8.2021.pdf
JSON decode error for https://www.doria.fi/handle/10024/181709 / https://www.doria.fi/bitstream/handle/10024/181709/Nils Villstrand ÅBO AKADEMI I SIN BÖRJAN doria 2021.pdf
CPU times: user 1.91 s, sys: 2.69 s, total: 4.61 s
Wall time: 2min 47s


In [3]:
# Analyze the extracted metadata

import json
import pandas as pd
import Levenshtein

records_meteor = []

ALMOST_THRESHOLD = 0.95  # similarity threshold to be considered "almost correct"

LANGMAP = {
    'fin': 'fi',
    'swe': 'sv',
    'eng': 'en'
}

FIELDS = {
    'year': 'dc.date.issued',
    'language': 'dc.language.iso',
    'title': 'dc.title',
    'publisher': 'dc.publisher',
    'authors': 'dc.contributor.author',
    'isbn': 'dc.identifier.isbn',
    'issn': 'dc.relation.eissn'
}

with open('test-records-meteor.jsonl') as infile:
    for line in infile:
        rec = json.loads(line)
        records_meteor.append(rec)

def compare_authors(rec):
    true_authors = set(rec.get('dc.contributor.author', []))
    try:
        predicted_authors = set([f"{author['lastname']}, {author['firstname']}"
                                for author in rec['meteor_output']['authors']])
    except (KeyError, TypeError):
        predicted_authors = set()

    if not true_authors and not predicted_authors:
        return ('not-relevant', 1)
    elif not true_authors:
        return ('found-nonexistent', 0)
    elif not predicted_authors:
        return ('not-found', 0)
    elif true_authors == predicted_authors:
        return ('exact', 1)
    elif true_authors.issubset(predicted_authors):
        return ('superset', 1)
    elif true_authors.issuperset(predicted_authors):
        return ('subset', 0)
    elif true_authors.intersection(predicted_authors):
        return ('overlap', 0)
    else:
        return ('wrong', 0)

def meteor_prediction(rec, meteor_key):
    try:
        return str(rec['meteor_output'][meteor_key]['value'])
    except (KeyError, TypeError):
        return None

def compare(rec, dc_key, meteor_key):
    
    true_val = rec.get(dc_key)

    # special case for "authors" field which may contain multiple values
    if dc_key == 'dc.contributor.author':
        return compare_authors(rec)
    
    # field-specific adjustments
    if dc_key == 'dc.language.iso':
        true_val = LANGMAP[true_val]  # convert to ISO 639-1 2-letter language code
    elif dc_key == 'dc.date.issued' and true_val is not None:
        true_val = true_val[:4]  # compare only the year
    elif dc_key == 'dc.identifier.isbn' and true_val:
        true_val = true_val[0]  # compare only against first (usually only) ISBN
        true_val = true_val.replace('-', '') # strip dashes in ISBNs
    elif dc_key == 'dc.publisher' and true_val:
        true_val = true_val[0]  # compare only against first (usually only) publisher

    predicted_val = meteor_prediction(rec, meteor_key)

    if predicted_val is None and true_val is None:
        return ('not-relevant', 1)
    elif predicted_val == true_val:
        return ('exact', 1)
    elif predicted_val is None:
        return ('not-found', 0)
    elif dc_key == 'dc.relation.eissn' and predicted_val == rec.get('dc.relation.pissn'):
        return ('printed-issn', 1)
    elif dc_key == 'dc.identifier.isbn' and predicted_val == rec.get('dc.relation.isbn', [''])[0].replace('-', ''):
        return ('related-isbn', 1)
    elif true_val is None:
        return ('found-nonexistent', 0)
    elif true_val in predicted_val:
        return ('superset', 1)
    elif true_val.lower() == predicted_val.lower():
        return ('case', 1)
    elif true_val.lower() in predicted_val.lower():
        return ('superset-case', 1)
    elif Levenshtein.ratio(true_val, predicted_val) >= ALMOST_THRESHOLD:
        return ('almost', 1)
    elif Levenshtein.ratio(true_val.lower(), predicted_val.lower()) >= ALMOST_THRESHOLD:
        return ('almost-case', 1)
    else:
        #if meteor_key not in ('title', 'language', 'year'):
        #if meteor_key == 'issn':
        #    print(rec['id'], meteor_key, repr(true_val), repr(predicted_val))
        return ('wrong', 0)

results = []

for rec in records_meteor:
    for meteor_field, dc_field in FIELDS.items():
        match_type, score = compare(rec, dc_field, meteor_field)
        results.append({
            'rowid': rec['rowid'],
            'language': rec['dc.language.iso'],
            'field': meteor_field,
            'prediction': meteor_prediction(rec, meteor_field),
            'true_val': rec.get(dc_field),
            'match_type': match_type,
            'score': score
        })

df = pd.DataFrame(results)
df

,rowid,language,field,prediction,true_val,match_type,score
0,serial4,eng,year,2019,2019-02-15,exact,1
1,serial4,eng,language,en,eng,exact,1
2,serial4,eng,title,Bank of Finland Research Discussion Papers 4 •...,Model-based regulation and firms' access to fi...,superset,1
3,serial4,eng,publisher,None,None,not-relevant,1
4,serial4,eng,authors,None,"[Tuuli, Saara]",exact,1
...,...,...,...,...,...,...,...
5070,mono294,eng,title,None,Constellations,not-found,0
5071,mono294,eng,publisher,None,[Academy of Fine Arts of the University of the...,not-found,0
5072,mono294,eng,authors,None,"[Feehily, Fergus, Suutari, Inkeri, Demozay, An...",not-found,0
5073,mono294,eng,isbn,9789527131510,[978-952-7131-51-0],exact,1


In [4]:
df.groupby(['language','field'])['score'].mean()

language  field    
eng       authors      0.608856
          isbn         0.878229
          issn         0.892989
          language     0.977860
          publisher    0.036900
          title        0.619926
          year         0.808118
fin       authors      0.578231
          isbn         0.918367
          issn         0.867347
          language     0.948980
          publisher    0.119048
          title        0.506803
          year         0.812925
swe       authors      0.668750
          isbn         0.918750
          issn         0.862500
          language     0.868750
          publisher    0.106250
          title        0.325000
          year         0.693750
Name: score, dtype: float64

In [5]:
value_counts_df = df.groupby(['language', 'field','score'])['match_type'].value_counts()
print(value_counts_df.to_string())

language  field      score  match_type       
eng       authors    0      not-found             61
                            wrong                 34
                            overlap                5
                            found-nonexistent      4
                            subset                 2
                     1      exact                106
                            superset              47
                            not-relevant          12
          isbn       0      not-found             17
                            found-nonexistent     10
                            wrong                  6
                     1      not-relevant         117
                            exact                 98
                            related-isbn          23
          issn       0      wrong                 22
                            not-found              6
                            found-nonexistent      1
                     1      not-relevant         172


In [6]:
pd.set_option('display.max_rows', None)
df[df['match_type'] == 'wrong'][df['field'] == 'isbn']

/tmp/ipykernel_74914/1478573037.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['match_type'] == 'wrong'][df['field'] == 'isbn']


,rowid,language,field,prediction,true_val,match_type,score
19,serial20,eng,isbn,9789526699639,[978-952-6699-74-5],wrong,0
40,serial32,eng,isbn,9789524627849,[978-952-462-785-6],wrong,0
215,serial111,eng,isbn,9789524620000,[978-952-6699-55-4],wrong,0
243,serial131,eng,isbn,9789526699639,[978-952-6699-70-7],wrong,0
2126,docthes46,eng,isbn,9515630452,[978-952-389-033-6],wrong,0
2532,docthes345,swe,isbn,9521234729,[978-951-765-993-2],wrong,0
3876,docthes838,swe,isbn,9789186717032,[978-952-389-004-6],wrong,0
4674,serial144,eng,isbn,9789523230880,[978-952-323-114-6],wrong,0
